## Google Colab utils
## *Don't forget to set GPU*

In [ ]:
!pip install keras-tuner

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd "/gdrive/My Drive/air-pollution"

## Modeling

In [ ]:
import warnings
warnings.filterwarnings('ignore')

from models import *
from kerastuner.tuners import RandomSearch

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
train_encoder_input_data = np.load('./data/third-order/Centar/train_encoder_input_data.npy')
train_decoder_input_data = np.load('./data/third-order/Centar/train_decoder_input_data.npy')
train_decoder_target_data = np.load('./data/third-order/Centar/train_decoder_target_data.npy')

valid_encoder_input_data = np.load('./data/third-order/Centar/valid_encoder_input_data.npy')
valid_decoder_input_data = np.load('./data/third-order/Centar/valid_decoder_input_data.npy')
valid_decoder_target_data = np.load('./data/third-order/Centar/valid_decoder_target_data.npy')

In [ ]:
print(train_encoder_input_data.shape)
print(train_decoder_input_data.shape)
print(train_decoder_target_data.shape)

In [ ]:
Tx, encoder_input_dim = (train_encoder_input_data.shape[1], 
                         train_encoder_input_data.shape[2])
    
Ty, decoder_input_dim = (train_decoder_input_data.shape[1], 
                         train_decoder_input_data.shape[2])

decoder_output_dim = train_decoder_target_data.shape[2]

In [ ]:
batch_size = 64
epochs = 150
max_trials = 1500
executions_per_trial = 1
patience = 15

### Standard

In [ ]:
model_builder = StandardSeq2Seq(Tx, Ty, encoder_input_dim, 
                             decoder_input_dim, decoder_output_dim)

tuner = RandomSearch(model_builder,
                     objective='val_loss',
                     max_trials=max_trials,
                     executions_per_trial=executions_per_trial,
                     directory='local-keras-tuner', 
                     project_name='standard')

In [ ]:
tuner.search(x=[train_encoder_input_data, 
                train_decoder_input_data], 
             y=train_decoder_target_data,
             validation_data=([
                valid_encoder_input_data,
                valid_decoder_input_data],
                valid_decoder_target_data),
             batch_size=batch_size,
             epochs=epochs,
             callbacks=[EarlyStopping(monitor='val_loss', 
                                      patience=patience, 
                                      verbose=1)]
            )

In [ ]:
tuner.results_summary()

### Attentive

In [ ]:
# due to the model architecture, we need to transform the output shape and type
train_attentive_decoder_target_data = list(np.swapaxes(
                                              train_decoder_target_data, 0, 1))
valid_attentive_decoder_target_data = list(np.swapaxes(
                                              valid_decoder_target_data, 0, 1))

In [ ]:
model_builder = AttentiveSeq2Seq(Tx, Ty, encoder_input_dim, decoder_input_dim,
                                decoder_output_dim)
tuner = RandomSearch(model_builder,
                     objective='val_loss',
                     max_trials=max_trials,
                     executions_per_trial=executions_per_trial,
                     directory='local-keras-tuner', 
                     project_name='attentive')

**If an epoch takes 3+ minutes, you probably forgot to set GPU :)**

In [ ]:
tuner.search(x=[train_encoder_input_data, 
                train_decoder_input_data], 
             y=train_attentive_decoder_target_data,
             validation_data=([
                valid_encoder_input_data,
                valid_decoder_input_data],
                valid_attentive_decoder_target_data),
             batch_size=batch_size,
             epochs=epochs,
             verbose=0,
             callbacks=[EarlyStopping(monitor='val_loss', 
                                      patience=patience, 
                                      verbose=1),
                        LossPrintingCallback(Ty)])

In [ ]:
tuner.results_summary()